In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d


In [2]:
def random_population(mu, d, low, high):
    pop = [(low + (high-low)*np.random.rand(d), np.ones(d).astype(float))
           for i in range(mu)]
    return np.array(pop)


def parents_selection(pop_v, lambd):
    if np.sum(np.abs(pop_v)) == 0:
        prop = np.ones(len(pop_v)) / len(pop_v)
    else:
        prop = np.abs(pop_v) / (np.sum(np.abs(pop_v)))  # normalization
    parents_idx = np.zeros(lambd)
    for i in range(lambd):
        parents_idx[i] = np.random.choice(
            np.arange(len(prop)), p=prop)  # roulette

    return parents_idx.astype(int)


def mutation(x, tau, tau0):
    x_v = x[0].copy()
    sigma_v = x[1].copy()
    eps0 = np.random.normal(0, tau0)
    eps_v = np.array([np.random.normal(0, tau) +
                      eps0 for i in range(len(sigma_v))])
    sigma_v = np.exp(eps_v) * sigma_v
    x_v = np.array([x + np.random.normal(0, sigma)
                    for (x, sigma) in zip(x_v, sigma_v)])

    return (x_v, sigma_v)


def best_idx(p_v, mu):
    return np.argpartition(p_v, mu)[:mu]


In [4]:
def run_instance(func, num_iterations=1000, d=30, K=0.2, mu=100, lambd=150, low=-20, high=20, plus=False):
    scores = np.zeros(num_iterations)
    p = random_population(mu, d, low, high)
    tau = K / np.sqrt(2*d)
    tau0 = K / np.sqrt(2*np.sqrt(d))
    for t in range(num_iterations):
        pop_v = np.array([func(x) for (x, sigma) in p])
        parents_idx = parents_selection(pop_v, lambd)
        children = []
        for i in range(len(parents_idx)):
            children.append(np.array(
                [p[parents_idx[:i+1]][:, 0].mean(axis=0), p[parents_idx[:i+1]][:, 1].mean(axis=0)]))
        children = np.array(children)
        children = np.array([mutation(c, tau, tau0) for c in children])
        if plus:
            candidates = np.concatenate((children, p[parents_idx]))
        else:
            candidates = children
        candidates[:, 0][candidates[:, 0] < low] = low
        candidates[:, 0][candidates[:, 0] > high] = high
        candidates_v = np.array([func(x) for (x, sigma) in candidates])
        p = candidates[best_idx(candidates_v, mu)]
        scores[t] = candidates_v.min()
    return scores


In [5]:
penalty = 10000
epsilon = 1e-2


def G3(x):
    ans = 0
    n = len(x)
    if abs(np.sum(x**2) - n) > epsilon:
        ans += penalty
    return ans-(np.sqrt(n)**n * np.prod(x))


def G13(x):
    ans = 0
    if abs(np.sum(x[:5]**2) - 10) > epsilon:
        ans = penalty
    if abs(x[1]*x[2] - 5*x[3]*x[4]) > epsilon:
        ans = penalty
    if abs(x[0]**3 + x[1]**3 + 1) > epsilon:
        ans = penalty
    return ans + np.exp(np.prod(x))


def G2(x):
    ans = 0
    if -np.prod(x) + 0.75 > 0:
        ans = penalty
    if np.sum(x) - 7.5*len(x) > 0:
        ans = penalty

    return ans-(abs((np.sum(np.cos(x)**4) - 2*np.prod(np.cos(x)**2))/np.sqrt(np.sum((np.arange(len(x)) + 1)*(x**2)))))


def G8(x):
    ans = 0
    if x[0]**2 - x[1] + 1 > 0:
        ans = penalty
    if 1 - x[0] + (x[1]-4)**2 > 0:
        ans = penalty

    return ans + (np.sin(2*np.pi*x[0])**3 * np.sin(2*np.pi*x[1]))/(x[0]**3 * (x[0] + x[1]))


def G11(x):
    ans = 0
    if abs(x[1]-x[0]**2) > epsilon:
        ans = penalty
    return x[0]**2 + (x[1] + 1)**2 + ans


In [6]:
instances_2 = [
    'G2 opt: 0.803619', (G2, 1000, 20, 0.25, 1500, 3000, 0.0001, 10, True),
    'G3 opt: 1', (G3, 1000, 10, 0.25, 1500, 3000, 0, 1, True),
    'G8 opt: 0.095825', (G8, 1000, 2, 0.2, 1500, 3000, 0.5, 5, True),
    'G11 opt: 0.75', (G11, 1000, 2, 0.25, 1500, 3000, -1, 1, True),
    'G13 opt: 0.0539498', (G13, 1000, 5, 0.25, 1500, 3000, -2, 2, True),
]
costs_inst_2 = []


In [ ]:
for i in instances_2:
    if type(i) != str:
        costs = run_instance(i[0], i[1], i[2], i[3],
                             i[4], i[5], i[6], i[7], i[8])
        print(i, costs.min())
        plt.figure()
        plt.plot(costs)
        plt.show()
    else:
        print(i)
        costs_inst_2.append(i)
